In [6]:
import numpy as np
import pandas as pd
import scipy.sparse as sparse
from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k, recall_at_k
from pathlib import Path


In [18]:
#число потоков нашего процессора
NUM_THREADS = 1 

#число параметров вектора 
NUM_COMPONENTS = 30 

#число эпох обучения
NUM_EPOCHS = 10 

In [7]:
ratings = pd.read_csv(Path.cwd() / 'data' / 'ratings.csv')
books = pd.read_csv(Path.cwd() / 'data' / 'books.csv')
tags = pd.read_csv(Path.cwd() / 'data' / 'tags.csv')
book_tags = pd.read_csv(Path.cwd() / 'data' / 'book_tags.csv')

In [9]:
mapper = dict(zip(books.goodreads_book_id,books.book_id))

In [16]:
tags = pd.read_csv(Path.cwd() / 'data' / 'tags_cleaned.csv')
book_tags = book_tags[book_tags.tag_id.isin(tags.tag_id)]
book_tags['id'] = book_tags.goodreads_book_id.apply(lambda x: mapper[x])

In [17]:
ratings_coo = sparse.coo_matrix((ratings.rating,(ratings.user_id,ratings.book_id)))
feature_ratings  = sparse.coo_matrix(([1]*len(book_tags),(book_tags.id,book_tags.tag_id)))

In [19]:
model = LightFM(learning_rate=0.05, loss='warp', no_components=NUM_COMPONENTS)

In [20]:
train,test = random_train_test_split(ratings_coo, test_percentage=0.2, random_state=None)

In [21]:
model = model.fit(train, epochs=NUM_EPOCHS, num_threads=NUM_THREADS,item_features =feature_ratings)

In [22]:
prec_score = precision_at_k(
                     model,
                     test,
                     num_threads=NUM_THREADS,
                     k=10,
                     item_features=feature_ratings).mean()

In [23]:
prec_score

0.082981125

In [24]:
recall_at_k = recall_at_k(model,
                     test,
                     num_threads=NUM_THREADS,
                     k=10,
                     item_features=feature_ratings).mean()

print(recall_at_k,prec_score)

0.038329028137034454 0.082981125
